# Working With Dates And Timestamps To Use With MongoDB

Go over ways to have your MongoDB entries reflect date and time

In [2]:
import bson
import time
import pytz
from datetime import datetime, timedelta

from pymongo import MongoClient

### Initalize A Client, Connect To DB, Speicfy Collection

In [3]:
client = MongoClient("mongodb://localhost:27017/")
db = client["my_store"]
users_col = db["users"]

### Looking at timestamp and datetime objects

In [23]:
time.time()

1727693066.0496657

In [5]:
type(time.time())

float

In [24]:
datetime.now()

datetime.datetime(2024, 9, 30, 11, 44, 46, 278434)

In [25]:
datetime.now() + timedelta(days=31)

datetime.datetime(2024, 10, 31, 11, 45, 5, 238234)

### UTC time

In [34]:
from datetime import timezone
datetime.now(timezone.utc)

datetime.datetime(2024, 9, 30, 12, 8, 17, 969876, tzinfo=datetime.timezone.utc)

In [31]:
time.timezone

0

In [32]:
datetime.fromtimestamp(time.time())

datetime.datetime(2024, 9, 30, 11, 48, 5, 208919)

### Inserting datetime and timestemp entries

In [41]:
users_col.insert_one(
    {
        "name": "Rat",
        "join_date_utc": datetime.now(timezone.utc),
        "join_timestamp": time.time() + time.timezone,
        "subscription_end_date_utc": datetime.now(timezone.utc)
    }
)

InsertOneResult(ObjectId('66fadbb1fc9db1a87d3758e5'), acknowledged=True)

### Querying and comparing datetime entries from MongoDB

In [42]:
users_col.find_one({"join_date_utc": {"$gt": datetime(2000,1,1)}})

{'_id': ObjectId('66fadb3cfc9db1a87d3758e1'),
 'name': 'Ethan',
 'join_date_utc': datetime.datetime(2024, 9, 30, 17, 9, 16, 113000),
 'join_timestamp': 1727716156.1131892,
 'subscription_end_date_utc': datetime.datetime(2024, 9, 30, 17, 9, 16, 113000)}

In [47]:
find_cursor = users_col.find().sort("join_date_utc", -1)

In [48]:
for document in find_cursor:
    print(document)
    print("\n")

{'_id': ObjectId('66fadbb1fc9db1a87d3758e5'), 'name': 'Rat', 'join_date_utc': datetime.datetime(2024, 9, 30, 17, 11, 13, 72000), 'join_timestamp': 1727716273.0724335, 'subscription_end_date_utc': datetime.datetime(2024, 9, 30, 17, 11, 13, 72000)}


{'_id': ObjectId('66fadbabfc9db1a87d3758e4'), 'name': 'Cat', 'join_date_utc': datetime.datetime(2024, 9, 30, 17, 11, 7, 19000), 'join_timestamp': 1727716267.0190182, 'subscription_end_date_utc': datetime.datetime(2024, 9, 30, 17, 11, 7, 19000)}


{'_id': ObjectId('66fadba3fc9db1a87d3758e3'), 'name': 'Bat', 'join_date_utc': datetime.datetime(2024, 9, 30, 17, 10, 59, 314000), 'join_timestamp': 1727716259.314219, 'subscription_end_date_utc': datetime.datetime(2024, 9, 30, 17, 10, 59, 314000)}


{'_id': ObjectId('66fadb9efc9db1a87d3758e2'), 'name': 'Matt', 'join_date_utc': datetime.datetime(2024, 9, 30, 17, 10, 54, 245000), 'join_timestamp': 1727716254.2452323, 'subscription_end_date_utc': datetime.datetime(2024, 9, 30, 17, 10, 54, 245000)}


{'

### Manually specifying BSON Timestamp

In [51]:
users_col.insert_one(
    {
        "name": "Frank",
        "join_due_time": datetime.now(timezone.utc),
        "join_timestamp":bson.Timestamp(int(time.time() + time.timezone), 1),
        "subscription_end_date_utc": datetime.now(timezone.utc) + timedelta(days=31)
    }
)

InsertOneResult(ObjectId('66faddd3fc9db1a87d3758e6'), acknowledged=True)

In [52]:
users_col.find_one({"name": "Frank"})

{'_id': ObjectId('66faddd3fc9db1a87d3758e6'),
 'name': 'Frank',
 'join_due_time': datetime.datetime(2024, 9, 30, 17, 20, 19, 690000),
 'join_timestamp': Timestamp(1727716819, 1),
 'subscription_end_date_utc': datetime.datetime(2024, 10, 31, 17, 20, 19, 690000)}

In [53]:
users_col.find_one({"name": "Frank"})["join_timestamp"].time

1727716819

### Converting UTC to local timezone 

In [54]:
find_one_res = users_col.find_one({})
find_one_res["join_date_utc"]

datetime.datetime(2024, 9, 30, 17, 9, 16, 113000)

In [57]:
pytz.utc.localize(find_one_res["join_date_utc"]).astimezone(pytz.timezone('GB'))

datetime.datetime(2024, 9, 30, 18, 9, 16, 113000, tzinfo=<DstTzInfo 'GB' BST+1:00:00 DST>)

In [56]:
print(pytz.all_timezones)

['Africa/Abidjan', 'Africa/Accra', 'Africa/Addis_Ababa', 'Africa/Algiers', 'Africa/Asmara', 'Africa/Asmera', 'Africa/Bamako', 'Africa/Bangui', 'Africa/Banjul', 'Africa/Bissau', 'Africa/Blantyre', 'Africa/Brazzaville', 'Africa/Bujumbura', 'Africa/Cairo', 'Africa/Casablanca', 'Africa/Ceuta', 'Africa/Conakry', 'Africa/Dakar', 'Africa/Dar_es_Salaam', 'Africa/Djibouti', 'Africa/Douala', 'Africa/El_Aaiun', 'Africa/Freetown', 'Africa/Gaborone', 'Africa/Harare', 'Africa/Johannesburg', 'Africa/Juba', 'Africa/Kampala', 'Africa/Khartoum', 'Africa/Kigali', 'Africa/Kinshasa', 'Africa/Lagos', 'Africa/Libreville', 'Africa/Lome', 'Africa/Luanda', 'Africa/Lubumbashi', 'Africa/Lusaka', 'Africa/Malabo', 'Africa/Maputo', 'Africa/Maseru', 'Africa/Mbabane', 'Africa/Mogadishu', 'Africa/Monrovia', 'Africa/Nairobi', 'Africa/Ndjamena', 'Africa/Niamey', 'Africa/Nouakchott', 'Africa/Ouagadougou', 'Africa/Porto-Novo', 'Africa/Sao_Tome', 'Africa/Timbuktu', 'Africa/Tripoli', 'Africa/Tunis', 'Africa/Windhoek', 'Ameri